# StableDiffusion NovelAi Google云盘安装版

第二次及以后：一次点击每个单元格进行部署

Credits: https://t.me/StableDiffusion_CN

https://github.com/AUTOMATIC1111/stable-diffusion-webui

https://github.com/TheLastBen/fast-stable-diffusion


In [ ]:
#@title 1、授权挂载Google云盘
from google.colab import drive

#@markdown 配置stable-diffusion-webui所在文件夹名称
SD_Folder_Name='sd' #@param {type:"string"}

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

drive.mount('/content/gdrive')
!mkdir -p /content/gdrive/MyDrive/$SD_Folder_Name
sd_dir="/content/gdrive/MyDrive/"+SD_Folder_Name
webui_dir="/content/gdrive/MyDrive/"+SD_Folder_Name+"/stable-diffusion-webui"

In [ ]:
#@title 1.1、安装插件和模型文件【可选】
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput

#@markdown 安装动画插件
deforum=False #@param {type:"boolean"}
#@markdown 安装momofuzi模型（4G大小注意云盘空间）
momofuzi=True #@param {type:"boolean"}
#@markdown 安装model模型（4G大小注意云盘空间）
model=True #@param {type:"boolean"}
%cd /content/
if deforum:
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui/ /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/deforum
if momofuzi:
  !gdown --id '1uoBjv7czngisMsdBPgn2PEOyveiqwjkZ' --output /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/momofuzi.ckpt
if model:
  !gdown --id '1UKxfl-b4NWh-A4GMV7K6Sod5IeoNVrwh' --output /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt 
!echo done

In [ ]:
#@title 2、初始化临时运行环境【每次新开实例执行一次】
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput

#安装环境依赖

#@markdown 启用快速安装（程序启动有问题时尝试重开实例并取消勾选）
Fast_Install_Dependencies=True #@param {type:"boolean"}
%cd /content/
if Fast_Install_Dependencies and '3.7' in sys.version:
  with capture.capture_output() as cap:
    !wget -O Dependencies_AUT.7z.001 https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.1
    !wget -O Dependencies_AUT.7z.002 https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/Dependencies_AUT.2
    !7z x Dependencies_AUT.7z.001
    !cp -r /content/usr/local/lib/python3.7/dist-packages /usr/local/lib/python3.7/
    !rm -r /content/usr
    !rm Dependencies_AUT.7z.001 Dependencies_AUT.7z.002
else:
  # Install triton
  !pip install -q -U --pre triton

if not os.path.exists('/tools/node/bin/lt'):
  !npm install -g localtunnel
!pip install -q gradio==3.5

gpuinfo = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')
# 安装cpu版tensorflow 避免deepdanbooru与sd冲突
!echo install tensorflow-cpu...
with capture.capture_output() as cap:
  !pip uninstall -y tensorflow-cpu tensorflow
  !pip install -q tensorflow-cpu tensorflow-io
!pip install -q git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru
!pip install -q scikit-image --upgrade

# Install xformers for python version < 3.10
if int(sys.version.split(' ')[0].split('.')[1])<10:
  !echo "$gpuinfo"
  if 'T4' in gpuinfo:
    if '3.7' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python37/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
    elif '3.8' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    elif '3.9' in sys.version:
      %pip install -q https://github.com/daswer123/xformers_prebuild_wheels/raw/main/Google%20Collab/T4/python39/xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl
    else:
      !echo "install xformers failed: unsupported python version."
  elif 'P100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'V100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl
  elif 'A100' in gpuinfo:
    %pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl
  else:
    print('it seems that your GPU is not supported at the moment, install xformers failed.')

!echo done

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.386s
     |████████████████████████████████| 5.3 MB 6.7 MB/s 
install tensorflow-cpu...
     |████████████████████████████████| 13.5 MB 7.8 MB/s 
Tesla T4
     |████████████████████████████████| 94.0 MB 37 kB/s 
done


In [ ]:
#@title 4、运行stable_diffusion_webui
from IPython.utils import capture
from subprocess import getoutput
import os,sys,time

#@markdown 启动成功后会有一个以gradio.app或loca.lt结尾的链接，点开即可访问程序

#@markdown 加载VAE文件：
Use_VAE=True #@param {type:"boolean"}

#@markdown 通过localtunnel内网穿透服务访问：
Use_Localtunnel_Share=False #@param {type:"boolean"}

#@markdown 启用xformers优化显存性能（T4机型已兼容，<font color="#dd0000">其他机型若无法出图请关闭</font>）
Enable_Xformers=True #@param {type:"boolean"}

##@markdown 优化attention.py，提升出图性能<font color="#dd0000">（会使hypernetwork失效，需要时请关闭）</font>
#Use_Optimized_Attention=False #@param {type:"boolean"}

#@markdown 加载tag自动补全插件
Load_Tagcomplete=False #@param {type:"boolean"}

#@markdown 自定义启动参数
User_Cmd_Args='' #@param {type:"string"}

!mkdir -p $webui_dir/cache/huggingface $webui_dir/cache/torch
%rm -f /root/.cache/huggingface
%rm -f /root/.cache/torch
!ln -s $webui_dir/cache/huggingface /root/.cache/
!ln -s $webui_dir/cache/torch /root/.cache/

if os.path.exists(webui_dir+"/repositories"):
  %rm -rf $webui_dir/repositories
!mkdir -p /content/repositories
%rm -f $webui_dir/repositories
!ln -s /content/repositories $webui_dir
#if os.path.exists('/content/repositories/stable-diffusion')==False:
#  with capture.capture_output() as cap:
#    %cd /content/repositories
#    !git clone https://github.com/CompVis/stable-diffusion.git stable-diffusion
#    !git -C stable-diffusion checkout 69ae4b35e0a0f6ee1af8bb9a5d0016ccb27e36dc
#    !git clone https://github.com/CompVis/taming-transformers.git taming-transformers
#    !git -C taming-transformers checkout 24268930bf1dce879235a7fddd0b2355b84d7ea6
#if Use_Optimized_Attention:
#  !wget -O /content/repositories/stable-diffusion/ldm/modules/attention.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/attention.py
#else:
#  !wget -O /content/repositories/stable-diffusion/ldm/modules/attention.py https://raw.githubusercontent.com/CompVis/stable-diffusion/main/ldm/modules/attention.py

extra_cmd=''
grdir=getoutput('pip show gradio | grep -o "/.*"')+'/gradio'
if Use_Localtunnel_Share:
  %cd /content/
  !nohup lt --port 7860 > url.txt 2>&1 &
  time.sleep(2)
  srvurl=getoutput("grep -o 'https[^ ]*' /content/url.txt")
  !sed -i 's@            self.server_name = .*@            self.server_name = "{srvurl[8:]}"@' $grdir/blocks.py
  !sed -i 's@            self.server_port = .*@            self.server_port = 443@' $grdir/blocks.py
  !sed -i 's@            self.protocol = .*@            self.protocol = "https"@' $grdir/blocks.py
  !sed -i 's@    "PUBLIC_SHARE_TRUE".*@    "PUBLIC_SHARE_TRUE": "[32mConnected",@' $grdir/strings.py
else:
  extra_cmd=extra_cmd+' --share'
  !sed -i 's@            self.server_name = .*@            self.server_name = server_name@' $grdir/blocks.py
  !sed -i 's@            self.server_port = .*@            self.server_port = server_port@' $grdir/blocks.py
  !sed -i 's@            self.protocol = .*@            self.protocol = "https" if self.local_url.startswith("https") else "http"@' $grdir/blocks.py

%cd $webui_dir
!git checkout -- webui.py
!sed -i 's@gpu_call).*@gpu_call) \n        demo.queue(concurrency_count=111500)@' $webui_dir/webui.py
# hacks to load model in colab
#!git checkout -- ./modules/shared.py
!sed -i 's@weight_load_location =.*@weight_load_location = "cuda"@' $webui_dir/modules/shared.py

if Load_Tagcomplete:
  with capture.capture_output() as cap:
    %cp -f $webui_dir/tags/config.json /content/tagconfig.bak
    !wget -O /content/tagcomplete.zip https://github.com/DominikDoom/a1111-sd-webui-tagcomplete/releases/latest/download/release.zip
    !unzip -o -d $webui_dir/ /content/tagcomplete.zip
    %cp -f /content/tagconfig.bak $webui_dir/tags/config.json 
else:
  %rm -f $webui_dir/javascript/tagAutocomplete.js

# launch
if Enable_Xformers:
  extra_cmd=extra_cmd+" --xformers"
if Use_VAE:
  extra_cmd=extra_cmd+" --vae-path "+sd_dir+"/animevae.pt --no-half-vae"
extra_cmd=extra_cmd+' '+User_Cmd_Args
!COMMANDLINE_ARGS="--deepdanbooru --disable-safe-unpickle --gradio-debug --config config.yaml $extra_cmd" REQS_FILE="requirements.txt" python launch.py

/content/gdrive/MyDrive/sd/stable-diffusion-webui
Python 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
Commit hash: 30b1bcc64e67ad50c5d3af3a6fe1bd1e9553f34e
Cloning Stable Diffusion into repositories/stable-diffusion...
Cloning Taming Transformers into repositories/taming-transformers...
Cloning K-diffusion into repositories/k-diffusion...
Cloning CodeFormer into repositories/CodeFormer...
Cloning BLIP into repositories/BLIP...
Installing requirements for Web UI
Launching Web UI with arguments: --deepdanbooru --disable-safe-unpickle --gradio-debug --config config.yaml --share --xformers --vae-path /content/gdrive/MyDrive/sd/animevae.pt --no-half-vae
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Downloading: 100% 1.59G/1.59G [00:48<00:00, 35.0MB/s]
Loa